# Arrow Library

In previous videos (or in my Python Fundamentals course) I covered libraries such as `humanize` and `pytz` as well as Python's built-in datetime functionality.

Those libraries are great and can be used independently, but the `Arrow` library kind of puts all that together in a much simpler and powerful way.

I highly recommend this library - it will make your life much easier.

Let's dive into some of the main features of the Arrow library.

## Installation

The library is available [here](https://github.com/arrow-py/arrow) and their very good documentation is available [here](https://arrow.readthedocs.io/en/latest/index.html)

You can pip install it in your virtual env using:
```bash
pip install arrow
```

## Constructing Naive and Aware Datetimes

In [1]:
import datetime

import arrow
import pytz

Arrow can create aware and naive datetimes easily:

In [2]:
arrow.utcnow()

<Arrow [2024-03-19T05:23:49.309104+00:00]>

As you can see, this is slightly different from Python's built-in `utcnow()` function:

In [3]:
datetime.datetime.utcnow()

/var/folders/kg/20pf1n8j6pjg09bwxz6cgw0r0000gn/T/ipykernel_4475/194642955.py:1: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  datetime.datetime.utcnow()


datetime.datetime(2024, 3, 19, 5, 23, 49, 315457)

As you can see Python's `utcnow()` function generates a naive datetime object, while, Arrow generates an aware datetime object.

By the way, Python's `utcnow()` function has been deprecated, and you should instead use this:

In [4]:
datetime.datetime.now(datetime.UTC)

datetime.datetime(2024, 3, 19, 5, 23, 49, 319434, tzinfo=datetime.timezone.utc)

And this function does return an aware datetime object.

I don't know about you, but I find writing
```python
arrow.utcnow()
```

much simpler than writing
```python
datetime.datetime.now(datetime.UTC)
```

Now, Arrow objects can easily be converted to regular datetime objects:

In [5]:
now = arrow.utcnow()
now.datetime

datetime.datetime(2024, 3, 19, 5, 23, 49, 322622, tzinfo=tzutc())

And to get a naive datetime (i.e. dropping the timezone info), we use the `.naive` property:

In [6]:
now.naive

datetime.datetime(2024, 3, 19, 5, 23, 49, 322622)

You can also easily extract just the timezone info from an Arrow object:

In [7]:
now.tzinfo

tzutc()

You can also create Arrow objects using the same arguments as you would with `datetime`:

In [8]:
dt = arrow.Arrow(2024, 3, 18, 22, 0, 0)
dt

<Arrow [2024-03-18T22:00:00+00:00]>

As you can see, Arrow will assume that a datetime created without specifying a timezone is in UTC.

Speaking of timezones...

# Dealing With Timezones

Arrow can easily deal with timezones as well, in a more straightforward fashion than `pytz`.

In [9]:
dt = arrow.now("US/Central")
dt

<Arrow [2024-03-19T00:23:49.337257-05:00]>

Converting from one timezone to another is just as easy:

In [10]:
dt.to("US/Pacific")

<Arrow [2024-03-18T22:23:49.337257-07:00]>

If you've watched my videos or my courses, you now that my preferred approach to dealing with datetimes is to always transform any "incoming" datetimes into naive UTC datetimes, and only convert to some other timezone for "outgoing" data.

Doing this with Arrow is dead easy.

Let's say our "incoming" data is a string such as this:

In [11]:
dt_input = "2024-03-18T20:30:00-07:00"

We can convert that into a naive UTC datetime this way:

In [12]:
dt = arrow.get(dt_input).to("UTC").naive
dt

datetime.datetime(2024, 3, 19, 3, 30)

## Formatting Datetimes

As we know, Python's datetime objects support the `strftime` method, where we can supply a specific format to use, using a variety of possible tokens.

The tokens for Python's formatting can be found [here](https://docs.python.org/3/library/datetime.html#strftime-and-strptime-format-codes)

For example:

In [13]:
dt = pytz.timezone("America/Phoenix").localize(datetime.datetime(2024, 3, 5, 20, 30))
dt

datetime.datetime(2024, 3, 5, 20, 30, tzinfo=<DstTzInfo 'America/Phoenix' MST-1 day, 17:00:00 STD>)

In [14]:
dt.strftime("%b %d, %Y (%I:%M %p %Z)")

'Mar 05, 2024 (08:30 PM MST)'

I am unaware if there even is a format token we can use to output a non-zero padded day, or month for that matter).

Also, I don't know about you, but I constantly have to hunt down that list of tokens as I never remember them.

Arrow also has a formatting function, and utilizes a far more intuitive set of tokens.

These are available [here](https://arrow.readthedocs.io/en/latest/guide.html#supported-tokens)

In [15]:
dt = arrow.Arrow(2024, 3, 5, 20, 30, tzinfo=pytz.timezone("America/Phoenix"))
dt

<Arrow [2024-03-05T20:30:00-07:00]>

Let's output the same thing as above:

In [16]:
dt.format("MMM D, YYYY (h:mm A ZZZ)")

'Mar 5, 2024 (8:30 PM MST)'

To me, Arrow is way more intuitive than Python's implementation.

Firthermore, Python has a single "standard" ISO output formatting:

In [17]:
dt.datetime.isoformat()

'2024-03-05T20:30:00-07:00'

Arrow, on the other hand, has a few more, documented [here](https://arrow.readthedocs.io/en/latest/guide.html#built-in-formats)

For the plain ISO format Python provides, Arrow also supports it:

In [18]:
dt.isoformat()

'2024-03-05T20:30:00-07:00'

However, we can use alternate pre-define standards as well:

In [19]:
dt.format(arrow.FORMAT_COOKIE)

'Tuesday, 05-Mar-2024 20:30:00 MST'

In [20]:
dt.format(arrow.FORMAT_W3C)

'2024-03-05 20:30:00-07:00'

## Date Arithmetic

As we already know, we can perform various date arithmetic in Python (safest is to always use UTC aware or naive timestamps as Python historically has not handled date arithmetic with DST and timezones very well - maybe that has changed, but I wouldn't know!)

In [21]:
dt = datetime.datetime.now(datetime.UTC)
dt

datetime.datetime(2024, 3, 19, 5, 23, 49, 388313, tzinfo=datetime.timezone.utc)

In [22]:
dt + datetime.timedelta(days=3)

datetime.datetime(2024, 3, 22, 5, 23, 49, 388313, tzinfo=datetime.timezone.utc)

`timedelta` objects are limited to parameters such as weeks, days, hours, ..., milliseconds. No, years or months are supported.

Arrow also has a way to deal with shifting times, but which will also allow months and years.

In [23]:
dt = arrow.utcnow()
dt

<Arrow [2024-03-19T05:23:49.394109+00:00]>

In [24]:
dt.shift(months=1, days=3, years=1, hours=1, minutes=20)

<Arrow [2025-04-22T06:43:49.394109+00:00]>

And it even handles DST changes just fine. 

For example:

In [25]:
dt = arrow.now("US/Central")
dt

<Arrow [2024-03-19T00:23:49.400068-05:00]>

At this time of year, DST is active, as we can see this way:

In [26]:
dt.dst()

datetime.timedelta(seconds=3600)

If we had a datetime when DST was not active, we would see a timedelta of `0`:

In [27]:
arrow.get(datetime.datetime(2023, 12, 15, 3), "US/Central").dst()

datetime.timedelta(0)

Back to our example, and let's see how Arrow handles a DST change when we add `8` months to our date:

In [28]:
dt

<Arrow [2024-03-19T00:23:49.400068-05:00]>

In [29]:
dt.shift(months=8)

<Arrow [2024-11-19T00:23:49.400068-06:00]>

Notice how the UTC offset has changed!

## Floor, Ceiling

Ever needed to get the "start" time of a day?

Arrow, makes this super easy:

In [30]:
dt

<Arrow [2024-03-19T00:23:49.400068-05:00]>

In [31]:
dt.floor('day')

<Arrow [2024-03-19T00:00:00-05:00]>

Or maybe the start of the hour instead:

In [32]:
dt.floor('hour')

<Arrow [2024-03-19T00:00:00-05:00]>

or the start year:

In [33]:
dt.floor('year')

<Arrow [2024-01-01T00:00:00-06:00]>

Same goes with the "end" of day, month, etc.

In [34]:
dt.ceil('month')

<Arrow [2024-03-31T23:59:59.999999-05:00]>

Notice that it even supports DST changes.

## Spans

Another very common operation is to find a beginning and end datetime given some starting datetime and criteria.

For example, suppose we want to find the quarter that encloses some specific datetime.

In [35]:
dt

<Arrow [2024-03-19T00:23:49.400068-05:00]>

In [36]:
dt.span('quarter')

(<Arrow [2024-01-01T00:00:00-06:00]>,
 <Arrow [2024-03-31T23:59:59.999999-05:00]>)

Or maybe just the month that encloses our datetime:

In [37]:
dt.span('month')

(<Arrow [2024-03-01T00:00:00-06:00]>,
 <Arrow [2024-03-31T23:59:59.999999-05:00]>)

How easy was this!!

## Ranges

This is related to spans.

Often we have a start and end date, and we want to iterate in some step size (be it hourly, daily, etc) between those two dates.

Let's see how to do this:

In [38]:
start_dt = arrow.get(datetime.datetime(2023, 1, 10, 3, 30), "US/Central")
end_dt = arrow.get(datetime.datetime(2023, 1, 15, 0, 0), "US/Central")

In [39]:
for dt in arrow.Arrow.range('day', start_dt, end_dt):
    print(dt)

2023-01-10T03:30:00-06:00
2023-01-11T03:30:00-06:00
2023-01-12T03:30:00-06:00
2023-01-13T03:30:00-06:00
2023-01-14T03:30:00-06:00


Notice how it did not include the "last day" in this output - because the end time ealier than what would have been the last day.

In [40]:
start_dt = arrow.get(datetime.datetime(2023, 1, 10, 3, 30), "US/Central")
end_dt = arrow.get(datetime.datetime(2023, 1, 15, 5, 0), "US/Central")

for dt in arrow.Arrow.range('day', start_dt, end_dt):
    print(dt)

2023-01-10T03:30:00-06:00
2023-01-11T03:30:00-06:00
2023-01-12T03:30:00-06:00
2023-01-13T03:30:00-06:00
2023-01-14T03:30:00-06:00
2023-01-15T03:30:00-06:00


Now it's included...

Notice how it just kept the same time as the start time, and incremented days one by one.

If we wanted to, we could change to the start of each day by using the `floor()` and `ceil()` methods we just saw:

In [41]:
for dt in arrow.Arrow.range('day', start_dt.floor('day'), end_dt.ceil('day')):
    print(dt)

2023-01-10T00:00:00-06:00
2023-01-11T00:00:00-06:00
2023-01-12T00:00:00-06:00
2023-01-13T00:00:00-06:00
2023-01-14T00:00:00-06:00
2023-01-15T00:00:00-06:00


And we could actually iterate through each day, getting the start/end of each day in our iteration quite easily:

In [42]:
for dt in arrow.Arrow.range('day', start_dt.floor('day'), end_dt.ceil('day')):
    print(dt.floor('day'), "-", dt.ceil('day'))

2023-01-10T00:00:00-06:00 - 2023-01-10T23:59:59.999999-06:00
2023-01-11T00:00:00-06:00 - 2023-01-11T23:59:59.999999-06:00
2023-01-12T00:00:00-06:00 - 2023-01-12T23:59:59.999999-06:00
2023-01-13T00:00:00-06:00 - 2023-01-13T23:59:59.999999-06:00
2023-01-14T00:00:00-06:00 - 2023-01-14T23:59:59.999999-06:00
2023-01-15T00:00:00-06:00 - 2023-01-15T23:59:59.999999-06:00


But Arrow has this particular use case covered as well:

In [43]:
for dt in arrow.Arrow.span_range('day', start_dt, end_dt):
    print(dt[0], "-", dt[1])

2023-01-10T00:00:00-06:00 - 2023-01-10T23:59:59.999999-06:00
2023-01-11T00:00:00-06:00 - 2023-01-11T23:59:59.999999-06:00
2023-01-12T00:00:00-06:00 - 2023-01-12T23:59:59.999999-06:00
2023-01-13T00:00:00-06:00 - 2023-01-13T23:59:59.999999-06:00
2023-01-14T00:00:00-06:00 - 2023-01-14T23:59:59.999999-06:00
2023-01-15T00:00:00-06:00 - 2023-01-15T23:59:59.999999-06:00


And of course, I've only looked at `day` here, but you also have all the other frames available such as `year`, `quarter`, etc.

## Humanizing and Localization

Last thing i want to touch on is humanizing dates.

Let's start by creating some datetimes that are two hours in the future and to hours in the past.

In [55]:
dt_now = arrow.Arrow.utcnow()
dt_past = dt_now.shift(hours=-2)
dt_future = dt_now.shift(hours=2)

dt_past, dt_now, dt_future

(<Arrow [2024-03-19T03:24:07.958542+00:00]>,
 <Arrow [2024-03-19T05:24:07.958542+00:00]>,
 <Arrow [2024-03-19T07:24:07.958542+00:00]>)

For display purposes, we may want to provide a fuzzy approximate representation of these dates relative to now.

In [56]:
dt_past.humanize()

'2 hours ago'

In [57]:
dt_future.humanize()

'in an hour'

We can even specify what the relative time is (instead of just current time):

In [58]:
dt_past.humanize(dt_now)

'2 hours ago'

In [59]:
dt_future.humanize(dt_now)

'in 2 hours'

We can also change the granularity of the humanized string:

In [60]:
dt_future.humanize(granularity="minute")

'in 119 minutes'

or even using multiple granularities:

In [61]:
dt_future.humanize(granularity=["hour", "minute"])

'in an hour and 59 minutes'

and of course we can also specify the relative date:

In [62]:
dt_future.humanize(dt_now, granularity=["hour", "minute"])

'in 2 hours and 0 minutes'

Furthermore, humanizations support multiple locales.

For example:

In [63]:
dt_future.humanize(granularity=["hour", "minute"], locale="fr")

'dans une heure et 59 minutes'

In [64]:
dt_future.humanize(granularity=["hour", "minute"], locale="hi")

'एक घंटा 59 मिनट  बाद'

A full list of currently implemented locales can be seen [here](https://github.com/arrow-py/arrow/blob/master/arrow/locales.py)

or you can also get a list of them this way:

In [65]:
from arrow import locales

sorted(locales._locale_map.items(), key=lambda el: el[0])

[('af', arrow.locales.AfrikaansLocale),
 ('af-nl', arrow.locales.AfrikaansLocale),
 ('am', arrow.locales.AmharicLocale),
 ('am-et', arrow.locales.AmharicLocale),
 ('ar', arrow.locales.ArabicLocale),
 ('ar-ae', arrow.locales.ArabicLocale),
 ('ar-bh', arrow.locales.ArabicLocale),
 ('ar-dj', arrow.locales.ArabicLocale),
 ('ar-dz', arrow.locales.AlgeriaTunisiaArabicLocale),
 ('ar-eg', arrow.locales.ArabicLocale),
 ('ar-eh', arrow.locales.ArabicLocale),
 ('ar-er', arrow.locales.ArabicLocale),
 ('ar-iq', arrow.locales.LevantArabicLocale),
 ('ar-jo', arrow.locales.LevantArabicLocale),
 ('ar-km', arrow.locales.ArabicLocale),
 ('ar-kw', arrow.locales.ArabicLocale),
 ('ar-lb', arrow.locales.LevantArabicLocale),
 ('ar-ly', arrow.locales.ArabicLocale),
 ('ar-ma', arrow.locales.MoroccoArabicLocale),
 ('ar-mr', arrow.locales.MauritaniaArabicLocale),
 ('ar-om', arrow.locales.ArabicLocale),
 ('ar-ps', arrow.locales.LevantArabicLocale),
 ('ar-qa', arrow.locales.ArabicLocale),
 ('ar-sa', arrow.locales.A

## Conclusion

This library has even more features I don't cover here, so take a look at the docs and consider using Arrow in your next Python project that has to deal with dates and times.